In [ ]:
# General imports
import numpy as np

# Meshcat imports
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess

# Drake imports
from pydrake.all import ConnectMeshcatVisualizer, DiagramBuilder
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback

In [ ]:
# System set up
builder = DiagramBuilder()
station = builder.AddSystem(ManipulationStation())
station.SetupClutterClearingStation()
station.Finalize()

# Mesh cat setup
frames_to_draw = {"iiwa": {"iiwa_link_1", "iiwa_link_2", "iiwa_link_3", "iiwa_link_4", "iiwa_link_5", "iiwa_link_6", "iiwa_link_7"},
                  "gripper": {"body"}}
proc, zmq_url, web_url = start_zmq_server_as_subprocess()
meshcat = ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url,
    frames_to_draw=frames_to_draw,
    axis_length=0.3,
    axis_radius=0.01)

# Further system set up
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant = station.get_multibody_plant()
gripper = plant.GetBodyByName("body")

# Finish meshcat setup
def pose_callback(context):
    pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
meshcat.load()
MakeJointSlidersThatPublishOnCallback(station.get_multibody_plant(), meshcat, context, my_callback=pose_callback)